# Multilabel Classification for Event Extraction

## Code Requirements

Following is the package requirements for the notebook
- Pytorch 1.4.0
- catboost 0.23
- xgboost 1.0.2
- sklearn 0.21.3

## Section 1: Data Preprocessing

## Section 2: Modeling

## Section 3: Result Analysis